本文学习于论文《Abstractive Sentence Summarization with Attentive Recurrent Neural Networks》，作者:Sumit Chopra等

# 摘要: Abstract

生成式摘要(Abstractive)生成给定句子的较短版本，同时试图保留其含义。我们提出了一个条件循环神经网络(RNN)，它生成输入句子的摘要。通过新颖的基于卷积注意力机制的编码器提供调节，该编码器确保解码器关注每一生成步骤上的适当的输入单词。我们的模型仅依赖于学习到的特征，并且易于在大型数据集上以端到端的方式进行训练。我们的实验表明，该模型明显优于最近提出的在Gigaword语料库上的最先进方法，同时它在DUC2004共享任务上具有竞争力。

# 引言: Introduction

生成段落的精简版本同时保留其含义称为文本摘要。解决这一任务是迈向自然语言理解的重要一步。文本摘要通常可大致分为两类：抽取式摘要(Extractive)通过从原始文本中裁剪重要的片段并将它们组合在一起以形成连贯的摘要；生成式摘要(Abstractive)从头开始生成摘要，而不受限于重用原始文本中的短语。

在本文中，我们为生成式摘要问题提出了一个新颖的循环神经网络。受最近提出的机器翻译架构的启发(Bahdanau的《Learning Phrase Representations using RNN Encoder–Decoder for Statistical Machine Translation》)，我们的模型由条件循环神经网络组成，它充当解码器以生成输入句子的摘要，这非常类似于标准的循环语言模型。另外，在每个时间步，解码器还接受可调节的输入，该可调节的输入是编码器模块的输出。根据RNN的当前状态，编码器计算输入句子中的单词的分数。这些分数可以解释为输入文本上的软对齐(soft alignment)，它通知解码器应该关注输入句子中的哪一部分以生成下一个单词。**解码器和编码器**在由**句子-摘要对**组成的数据集上进行**联合训练**。我们的模型可以看作是Rush等人针对同一问题最近提出的模型的扩展。然而他们使用前馈神经语言模型进行生成，我们使用循环神经网络。此外，我们的编码器更复杂，因为它明确地编码输入单词的位置信息。最后，我们的编码器使用卷积网络对输入单词进行编码。这些扩展可以提高性能。

本文的主要贡献在于提出了一个新的基于卷积注意力机制的条件循环神经网络用于生成式摘要。从经验上说，我们证明了我们的模型在多个数据集上优于Rush等人提出的最先进技术。特别值得注意的是，即使不使用任何抽取式特征调整的简单生成模型，我们的模型也能够在Gigaword数据集上明显优于其ABS+系统，并且在DUC2004数据集上具有可比性。

# 先前工作: Previous Work

虽然有大量的关于抽取式摘要的研究，但对生成式摘要的研究却少得多。基于计数的含噪通道的机器翻译模型被提出来用于Banko 等人的问题。生成式摘要后来围绕DUC-2003和DUC-2004竞赛被正式化了，其中TOPIARY系统是最好的。最近Cohn和Lapata以及后来的Woodsend等人提出了大量**使用句子-摘要对的句法特征**的系统。后来，沿着Banko等人的路线，MOSES被Wubben等人直接用作文本简化的方法。

# 注意力循环结构: Attentive Recurrent Architecture

假设 $x$ 表示由 $M$ 个单词构成的输入序列，即 $x = \left[x_{1},...,x_{M}\right]$, 且每个单词 $x_{i}$ 是词包 $\nu$ 的一部分，词包大小为$|\nu| = V$。我们的任务是生成一个目标序列 $y = \left[y_{1},...,y_{N}\right]$, 序列含有 $N$ 个单词，且 $N < M$。这样通过下述方式就保留了序列 $x$ 的含义: $y = argmax_{y}P(y|x)$,其中 $y$ 是表示 $N$ 个单词序列的随机变量。

通常，条件概率由具有参数 $\theta$ 的参数函数建模：$P(y|x) = P(y|x;\theta)$ 。训练模型的方式便是，找到最大化训练语料中句子-摘要对条件概率的 $\theta$。如果模型被训练来生成摘要的下一个单词，给定前面的单词，则上述条件可以被分解为单个条件概率的乘积:

$$P(y|x;\theta) = \prod_{t=1}^{N}p\left(y_{t}|\left\{y_{1},...,y_{t-1}\right\},x;\theta\right) (1)$$

在本文中，我们受(Cho的《Learning phrase representations using RNN encoder-decoder for statistical machine translation》)启发，用RNN Enocder-Decoder结构对条件概率进行建模，并在(Bahdanau的《Neural machine translation by jointly learning to align and translate》)上做扩展。我们称自己的模型为$RAS$(Recurrent Attention Summarizer)。

## 循环解码器: Recurrent Decoder

上述条件使用RNN建模:

$$P(y_{t}|\left\{y_{1},...,y_{t-1}\right\},x;\theta) = P_{t} = g\theta_{1}(h_{t},c_{t}),$$

这里 $h_{t}$ 是RNN的隐藏状态：

$$h_{t} = g\theta_{1}(y_{t-1},h_{t-1},c_{t})$$

这里 $c_{t}$ 是编码器模块的输出(详细见注意力编码器部分)。它被视作上下文向量，由当前状态 $h_{t-1}$ 和 输入序列 $x$ 的函数计算而得。

我们的Elman RNN (Elman的《Finding structure in time》)采用以下形式:

$$h_{t} = \sigma\left(W_{1}y_{t-1} + W_{2}h_{t-1} + W_{3}c_{t}\right)$$

$$P_{t} = \rho\left(W_{4}h_{t} + W_{5}c_{t}\right)$$

这里 $\sigma$ 是 $sigmoid$ 函数，$\rho$ 是 $softmax$ 函数，定义为: $\rho(o_{t}) = e^{o_{t}}/\sum_{j}e^{o_{j}}$ ,$W_{i}(i = 1,...,5)$ 是可学习的参数矩阵，$W_{1,2,3} \in R^{d\times d}$,$W_{4,5} \in R^{d \times V}$。

LSTM Decoder 定义如下(S. Hochreiter的《Long shortterm memory. Neural Computation》)：

$$i_{t} = \sigma\left(W_{1}y_{t-1} + W_{2}h_{t-1} + W_{3}c_{t}\right)$$

$$i_{t}^{'} = tanh(W_{4}y_{t-1} + W_{5}h_{t-1} + W_{6}c_{t})$$

$$f_{t} = \sigma\left(W_{7}y_{t-1} + W_{8}h_{t-1} + W_{9}c_{t}\right)$$

$$o_{t} = \sigma\left(W_{10}y_{t-1} + W_{11}h_{t-1} + W_{12}c_{t}\right)$$

$$m_{t} = m_{t-1} \odot f_{t} + i_{t} \odot i_{t}^{'}$$

$$h_{t} = m_{t} \odot o_{t}$$

$$P_{t} = \rho\left(W_{13}h_{t} + W_{14}c_{t}\right)$$

操作符 $\odot$ 指各分量分别相乘，$W_{i}(i = 1,...,14)$是可学习的参数矩阵，$W_{1,...,12}\in R^{d \times d}$,$W_{13,14} \in R^{d \times V}$

## 注意力编码器: Attentive Encoder

现在，我们给出计算编码器的细节，它用来计算解码器阶段每一时间步的上下文向量$c_{t}$。对于输入序列 $x$ ，我们用 $x_{i}$ 表示第 $i$ 个单词($x_{i} \in R^{d}$)的 $d$ 维度的可学习的嵌入表示。此外，对于位置 $i$ 处的单词 $x_{i}$ 还与 $d$ 维度的可学习嵌入表示 $l_{i}$ ($l_{i} \in R^{d}$)相关。因此，位置 $i$ 处的单词 $x_{i}$ 的完整的嵌入表示如下: $\alpha_{i} = x_{i} + l_{i}$ 。我们用 $B^{k} \in R^{q \times d}$ 表示一个可学习权重矩阵，它用于在连续单词的完整嵌入表示上进行卷积。由$d$ 这样的矩阵$k \in \left\{1,...,d\right\}$ 。卷积的输出是:

$$z_{ik} = \sum_{h=-q/2}^{q/2} \alpha_{i+h}\cdot b_{q/2 + h}^{k}　　(2)$$

这里，$b_{j}^{k}$ 是矩阵 $B^{k}$ 的第 $j$ 列。因此，$d$ 维的聚合嵌入向量 $z_{i}$ 定义如下: $z_{i} = \left[z_{i1},...,z_{id}\right]$ 。值得注意的是，输入序列中每个单词 $x_{i}$ 与一个聚合嵌入向量 $z_{i}$ 相关联。向量 $z_{i}$ 可以看作是单词的表示，其捕获了单词在句子中出现的位置以及它所在句子的上下文。在我们的实验中，卷积矩阵 $B^{k}$ 的宽度设置为 $5$ 。考虑到输入序列 $x$ 边界处的单词，我们在计算聚合嵌入向量 $z_{i}$ 之前用 假单词(填充符号)填充输入序列的两侧。

当有单词的聚合嵌入向量后，我们以如下的方式计算编码器的输出，即上下文向量 $c_{t}$ ：

$$c_{t} = \sum_{j=1}^{M}\alpha_{j,t-1}x_{j}　，　(3)$$

其中，权重 $\alpha_{j,t-1}$ 为:

$$\alpha_{j,t-1} = \frac{exp(z_{j}\cdot h_{t-1})}{\sum_{i=1}^{M}exp(z_{i}\cdot h_{t-1})},　　(4)$$

## 训练和生成: Training and Generation

给定训练语料 $S = \left\{(x^{i},y^{i})\right\}_{i=1}^{S}$, $S$ 是 句子-摘要对，通过最小化训练数据与 $\theta$ 相关的 负的条件对数似然，我们的模型可以用随机梯度下降法以端到端地方式训练:

$$L = - \sum_{i=1}^{S} \sum_{t=1}^{N}log P(y_{t}^{i}|\left\{y_{1}^{i},...,y_{t-1}^{i}\right\},x^{i};\theta), (5)$$

其中， $\theta$ 表示解码器和编码器中的参数。

一旦训练完含参数的模型，我们通过基于单词的 beam-search 为新句子 $x$ 生成摘要,使得 $P(y|x)$ 最大化，即 $argmax P(y_{t}|\left\{y_{1},...,y_{t-1}\right\},x)$ 。beam-search 可以通过每个时间步保留的路径数 $k$ 来参数化搜索。

# 实验设置: Experimental Setup

## 数据集和评估: Datasets and Evaluation

我们的模型在Gigaword语料库的注释版本上训练，同时我们只使用注释来标记和分离句子，同时放弃其他注释，如标记和解析。我们将每篇文章的第一句与其标题配对以形成 句子-摘要对 。数据处理方式与 Rush(Alexander M Rush的《A neural attention model for abstractive sentence summarization》) 的一致，同时我们对训练集、验证集和测试机使用相同的分割。对于Gigaword，我们在2000个 句子-摘要对的相同的随机保留测试集上展示结果。我们还在包含500对的DUC2004数据集上评估模型。我们的评估基于ROUGE的三个变体，即ROUGE-1(unigrams),ROUGE-2(bigrams),ROUGE-L(longest-common substring)。

## 结构上的选择: Architectural Choices

模型的代码可以在[此处](www://github.com/facebook/namas)找到，我们用PyTorch实现。为了优化等式(5)的损失函数，我们用 `mini-batch=32` 的随机梯度下降。在训练期间，我们测量验证集的复杂度，并根据此调整炒菜时，例如学习率。

在解码器上，我们实验了RNN和LSTM结构。我们基于网格搜索来选择超参数，并选择在验证集上表现最好的那个。具体而言，我们搜索了循环层的隐藏单元 $H$ 的数量，学习率 $\eta$ ,学习率衰减参数 $\gamma$ 和梯度裁剪的阈值 $\kappa$ 。我们RNN结构使用单层的，且隐藏单元 $H = 512$ ，$\eta = 0.5$ ，$\gamma = 2$ ，$\kappa = 10$。我们LSTM结构使用单层的，且隐藏单元 $H = 512$ ，$\eta = 0.1$ ，$\gamma = 2$ ，$\kappa = 10$。

# 结论: Conclusion

我们将最好的Rush的模型(Alexander M Rush的《A neural attention model for abstractive sentence summarization》)扩展到了循环神经网络结构中。我们的模型是用于机器翻译Encoder-Decoder框架(Bahdanau的《Neural machine translation by jointly learning to align and translate》)的简化版本。该模型在Gigaword语料库上进行训练，以根据每篇新闻文章的第一行生成标题。尽管我们的模型不依赖于额外抽取的特征，但我们在Gigaword数据和DUC-2004挑战方面的表现都优于先前的最新技术水平。